In [1]:
%reset -f

In [5]:
import pandas as pd
from pathlib import Path
from sssom.parsers import parse_sssom_table

In [12]:
# Functions

def flip_predicate(predicate_id):
    flip_dict = {
        "skos:closeMatch": "skos:closeMatch",
        "skos:relatedMatch": "skos:relatedMatch",
        "skos:narrowMatch" : "skos:broadMatch",
        "skos:broadMatch" : "skos:narrowMatch",
        "skos:exactMatch" : "skos:exactMatch"
    }
    
    return flip_dict[predicate_id]

In [4]:
data_dir = Path("__file__").resolve().parents[1] / "data"
mondo_lexmatch_sssom = data_dir/ "mondo_lexmatch.sssom.tsv"

In [6]:
%%time
mondo_lexmatch_df = parse_sssom_table(mondo_lexmatch_sssom)
mondo_lexmatch_df.df.head()

CPU times: user 7min 20s, sys: 3.26 s, total: 7min 23s
Wall time: 7min 25s


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,<http://identifiers.org/hgnc/10263>,RP1,skos:closeMatch,MONDO:0008377,retinitis pigmentosa 1,semapv:LexicalMatching,oaklib,0.5,rdfs:label,oio:hasExactSynonym,rp1
1,<http://identifiers.org/hgnc/10274>,RP2,skos:closeMatch,MONDO:0010723,retinitis pigmentosa 2,semapv:LexicalMatching,oaklib,0.5,rdfs:label,oio:hasExactSynonym,rp2
2,<http://identifiers.org/hgnc/10274>,RP2,skos:closeMatch,MONDO:0100442,RP2-related retinopathy,semapv:LexicalMatching,oaklib,0.5,rdfs:label,oio:hasNarrowSynonym,rp2
3,<http://identifiers.org/hgnc/10288>,RP9,skos:closeMatch,MONDO:0008378,retinitis pigmentosa 9,semapv:LexicalMatching,oaklib,0.5,rdfs:label,oio:hasExactSynonym,rp9
4,<http://identifiers.org/hgnc/10457>,RS1,skos:closeMatch,MONDO:0010725,X-linked retinoschisis,semapv:LexicalMatching,oaklib,0.5,rdfs:label,oio:hasRelatedSynonym,rs1


In [9]:
condition_mondo_subj = mondo_lexmatch_df.df['subject_id'].str.contains("MONDO")
condition_mondo_obj = mondo_lexmatch_df.df['object_id'].str.contains("MONDO")

non_mondo_subjects_df =  pd.DataFrame(mondo_lexmatch_df.df[(~condition_mondo_subj & condition_mondo_obj)])
mondo_subjects_df = pd.DataFrame(mondo_lexmatch_df.df[(condition_mondo_subj & ~condition_mondo_obj)])

non_mondo_subjects_df.head()

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,<http://identifiers.org/hgnc/10263>,RP1,skos:closeMatch,MONDO:0008377,retinitis pigmentosa 1,semapv:LexicalMatching,oaklib,0.5,rdfs:label,oio:hasExactSynonym,rp1
1,<http://identifiers.org/hgnc/10274>,RP2,skos:closeMatch,MONDO:0010723,retinitis pigmentosa 2,semapv:LexicalMatching,oaklib,0.5,rdfs:label,oio:hasExactSynonym,rp2
2,<http://identifiers.org/hgnc/10274>,RP2,skos:closeMatch,MONDO:0100442,RP2-related retinopathy,semapv:LexicalMatching,oaklib,0.5,rdfs:label,oio:hasNarrowSynonym,rp2
3,<http://identifiers.org/hgnc/10288>,RP9,skos:closeMatch,MONDO:0008378,retinitis pigmentosa 9,semapv:LexicalMatching,oaklib,0.5,rdfs:label,oio:hasExactSynonym,rp9
4,<http://identifiers.org/hgnc/10457>,RS1,skos:closeMatch,MONDO:0010725,X-linked retinoschisis,semapv:LexicalMatching,oaklib,0.5,rdfs:label,oio:hasRelatedSynonym,rs1


In [14]:
desired_sequence = ['subject_id', 'subject_label', 'predicate_id', 'object_id',
       'object_label', 'mapping_justification', 'mapping_tool', 'confidence',
       'subject_match_field', 'object_match_field', 'match_string']

new_subjects_df = non_mondo_subjects_df.rename(columns={
    'subject_id':'object_id', 
    'subject_label':'object_label',
    'object_id':'subject_id', 
    'object_label':'subject_label',
    'subject_match_field': 'object_match_field',
    'object_match_field': 'subject_match_field'
    
})


new_subjects_df = new_subjects_df[desired_sequence]
new_subjects_df["predicate_id"] = new_subjects_df["predicate_id"].apply(lambda x: flip_predicate(x))
print(len(new_subjects_df))
new_subjects_df.head()

1177


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,MONDO:0008377,retinitis pigmentosa 1,skos:closeMatch,<http://identifiers.org/hgnc/10263>,RP1,semapv:LexicalMatching,oaklib,0.5,oio:hasExactSynonym,rdfs:label,rp1
1,MONDO:0010723,retinitis pigmentosa 2,skos:closeMatch,<http://identifiers.org/hgnc/10274>,RP2,semapv:LexicalMatching,oaklib,0.5,oio:hasExactSynonym,rdfs:label,rp2
2,MONDO:0100442,RP2-related retinopathy,skos:closeMatch,<http://identifiers.org/hgnc/10274>,RP2,semapv:LexicalMatching,oaklib,0.5,oio:hasNarrowSynonym,rdfs:label,rp2
3,MONDO:0008378,retinitis pigmentosa 9,skos:closeMatch,<http://identifiers.org/hgnc/10288>,RP9,semapv:LexicalMatching,oaklib,0.5,oio:hasExactSynonym,rdfs:label,rp9
4,MONDO:0010725,X-linked retinoschisis,skos:closeMatch,<http://identifiers.org/hgnc/10457>,RS1,semapv:LexicalMatching,oaklib,0.5,oio:hasRelatedSynonym,rdfs:label,rs1


In [17]:
lex_df = pd.concat([mondo_subjects_df,new_subjects_df], ignore_index=True)
print(len(lex_df))

lex_df.head()

1254


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,MONDO:0000170,"microphthalmia, isolated, with coloboma",skos:closeMatch,NCBITaxon:120793,Mycobacterium avium complex (MAC),semapv:LexicalMatching,oaklib,0.5,oio:hasExactSynonym,oio:hasBroadSynonym,mac
1,MONDO:0001103,giardiasis,skos:closeMatch,NCBITaxon:5740,Giardia <diplomonads>,semapv:LexicalMatching,oaklib,0.5,oio:hasRelatedSynonym,oio:hasExactSynonym,giardia
2,MONDO:0001191,hirudiniasis,skos:closeMatch,NCBITaxon:55824,Hirudinea,semapv:LexicalMatching,oaklib,0.5,oio:hasExactSynonym,oio:hasExactSynonym,leeches
3,MONDO:0001370,pericardial effusion,skos:closeMatch,UBERON:0002409,pericardial fluid,semapv:LexicalMatching,oaklib,0.5,oio:hasExactSynonym,rdfs:label,pericardial fluid
4,MONDO:0001794,Pthirus pubis infestation,skos:closeMatch,NCBITaxon:121228,Pthirus pubis,semapv:LexicalMatching,oaklib,0.5,oio:hasExactSynonym,oio:hasRelatedSynonym,phthirus pubis
